In [ ]:
%pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from prophet import Prophet

from sklearn.metrics import mean_squared_error, mean_absolute_error

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import warnings
warnings.filterwarnings("ignore")

plt.style.use("ggplot")
plt.style.use("fivethirtyeight")
color_pal = sns.color_palette()

def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates MAPE given y_true and y_pred.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Load Data

In [ ]:
pjme = pd.read_csv("hourly-energy-consumption/PJME_hourly.csv",
                   index_col=[0],
                   parse_dates=[0])
pjme = pjme.sort_index()
pjme

In [ ]:
pjme.plot(style=".",
          figsize=(15,5),
          ms=1,
          color=color_pal[0],
          title="PJME MW")

In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()

    df["hour"] = df.index.hour
    df["dayofweek"] = df.index.dayofweek
    df["quarter"] = df.index.quarter
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["dayofyear"] = df.index.dayofyear
    df["season"] = df.index.month.map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                        3: 'Spring', 4: 'Spring', 5: 'Spring',
                                        6: 'Summer', 7: 'Summer', 8: 'Summer',
                                        9: 'Fall', 10: 'Fall', 11: 'Fall'})

    return df

In [ ]:
features_and_target = create_features(pjme)
features_and_target

In [ ]:
columns = ['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear']

for column in columns:
    fig, ax = plt.subplots(figsize=(15,5))

    sns.boxplot(data=features_and_target.dropna(),
                x=column,
                y="PJME_MW",
                hue="season",
                ax=ax,
                linewidth=1)
    ax.set_title(f"Power Use MW by {column}")
    ax.set_xlabel(column)
    ax.set_ylabel("Energy (MW)")
    ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Average consumption by hour across all days
hourly_avg = features_and_target.groupby('hour')['PJME_MW'].mean()
ax1.plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2, markersize=8, color=color_pal[0])
ax1.set_xlabel('Hour of Day', fontsize=12)
ax1.set_ylabel('Average Energy (MW)', fontsize=12)
ax1.set_title('Daily Seasonality: Average Energy Consumption by Hour', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(0, 24))

# Plot 2: One week of actual data to show daily pattern repeating
one_week = pjme['2015-01-01':'2015-01-07']
ax2.plot(one_week.index, one_week['PJME_MW'], linewidth=1.5, color=color_pal[1])
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Energy (MW)', fontsize=12)
ax2.set_title('Daily Pattern Over One Week (Jan 1-7, 2015)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Train/Test Split

In [ ]:
split_date = "1-Jan-2015"

pjme_train = pjme.loc[pjme.index <= split_date].copy()
pjme_test = pjme.loc[pjme.index > split_date].copy()

pjme_test \
    .rename(columns={"PJME_MW": "TEST SET"}) \
    .join(pjme_train.rename(columns={"PJME_MW": "TRAINING SET"}), how="outer") \
    .plot(figsize=(15,5), title="PJME TRAIN/TEST SPLIT", style=".", ms=1)

# Simple Prophet Model

* This model expects the data to be named a specific way.
    * **Datetime column:** ds
    * **target:** y

In [ ]:
pjme_train_prophet = pjme_train.reset_index().rename(columns={"Datetime": "ds", "PJME_MW": "y"})
pjme_test_prophet = pjme_test.reset_index().rename(columns={"Datetime": "ds", "PJME_MW": "y"})

In [ ]:
%%time

model = Prophet()
model.fit(pjme_train_prophet) 

In [ ]:
pjme_test_forecast = model.predict(pjme_test_prophet)

In [ ]:
pjme_test_forecast.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

fig = model.plot(pjme_test_forecast, ax=ax)
ax.set_title("Prophet Forecast")

In [ ]:
model.plot_components(pjme_test_forecast)
plt.show()

# Compare Forecasts to Actuals

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model.plot(pjme_test_forecast, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model.plot(pjme_test_forecast, ax=ax)

ax.set_xbound(lower=pd.Timestamp("2015-01-01"),
              upper=pd.Timestamp("2015-02-01"))
ax.set_ylim(0, 60000)

ax.set_title("January 2015 Forecasts vs Actuals")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model.plot(pjme_test_forecast, ax=ax)

ax.set_xbound(lower=pd.Timestamp("2015-01-01"),
              upper=pd.Timestamp("2015-01-08"))
ax.set_ylim(0, 60000)

ax.set_title("First Week of January 2015 Forecasts vs Actuals")
plt.show()

# Evaluate the Model

In [ ]:
rmse = np.sqrt(mean_squared_error(y_true=pjme_test["PJME_MW"],
                           y_pred=pjme_test_forecast["yhat"]))

print(f"Root Mean Squared Error: {rmse:0.4f}")

In [ ]:
mae = mean_absolute_error(y_true=pjme_test["PJME_MW"],
                          y_pred=pjme_test_forecast["yhat"])

print(f"Mean Absolute Error: {mae:0.4f}")

In [ ]:
off_percentage = mean_absolute_percentage_error(y_true=pjme_test["PJME_MW"],
                               y_pred=pjme_test_forecast["yhat"])

print(f"The model is {off_percentage:0.2f}% off on average.")

# Add Holidays

In [ ]:
cal = calendar()

holidays = cal.holidays(start=pjme.index.min(),
                        end=pjme.index.max(),
                        return_name=True)

holidays_df = pd.DataFrame(data=holidays,
                           columns=["holiday"])

holidays_df = holidays_df.reset_index().rename(columns={"index":"ds"})

In [ ]:
holidays_df.head()

In [ ]:
model_with_holidays = Prophet(holidays=holidays_df)

In [ ]:
%%time

model_with_holidays.fit(pjme_train_prophet)

In [ ]:
pjme_test_forecast_with_hols = model_with_holidays.predict(pjme_test_prophet)

In [ ]:
model_with_holidays.plot_components(pjme_test_forecast_with_hols)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model_with_holidays.plot(pjme_test_forecast_with_hols, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model_with_holidays.plot(pjme_test_forecast_with_hols, ax=ax)

ax.set_xbound(lower=pd.Timestamp("2015-01-01"),
              upper=pd.Timestamp("2015-02-01"))
ax.set_ylim(0, 60000)

ax.set_title("January 2015 Forecasts vs Actuals")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color="r")
fig = model_with_holidays.plot(pjme_test_forecast_with_hols, ax=ax)

ax.set_xbound(lower=pd.Timestamp("2015-07-01"),
              upper=pd.Timestamp("2015-07-08"))
ax.set_ylim(0, 60000)

ax.set_title("First Week of July (4th of July) 2015 Forecasts vs Actuals")
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(y_true=pjme_test["PJME_MW"],
                           y_pred=pjme_test_forecast_with_hols["yhat"]))

print(f"Root Mean Squared Error: {rmse:0.4f}")

In [ ]:
mae = mean_absolute_error(y_true=pjme_test["PJME_MW"],
                          y_pred=pjme_test_forecast_with_hols["yhat"])

print(f"Mean Absolute Error: {mae:0.4f}")

In [ ]:
off_percentage = mean_absolute_percentage_error(y_true=pjme_test["PJME_MW"],
                               y_pred=pjme_test_forecast_with_hols["yhat"])

print(f"The model is {off_percentage:0.2f}% off on average.")

# Predict into the Future

In [ ]:
future = model_with_holidays.make_future_dataframe(periods=365*24, include_history=False, freq="h")

In [ ]:
forecast = model_with_holidays.predict(future)

In [ ]:
forecast["yhat"]